## Import the dependencies

In [5]:
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F 
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision import datasets
from torchvision.transforms import v2 as transforms
from torchvision.ops import Conv2dNormActivation

from dataclasses import dataclass
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sn

import matplotlib.pyplot as plt
import time
import numpy as np
import random
import warnings
import os
from tqdm import tqdm

import pandas as pd

warnings.filterwarnings("ignore")

In [7]:
def set_seed():
    SEED_NUMBER = 42

    random.seed(SEED_NUMBER)
    np.random.seed(SEED_NUMBER)
    torch.manual_seed(SEED_NUMBER)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED_NUMBER)
        torch.cuda.manual_seed_all(SEED_NUMBER)

        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

set_seed()

## Download and Extract the Dataset

In [9]:
# Didn't work in vscode, so downloaded it by clicking on the link
#  "https://www.dropbox.com/s/45jdd8padeyjq6t/10_Monkey_Species.zip?dl=1" -O "10_Monkey_Species.zip"